In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
#Loading the neccessary libraries
import itertools
import torch
import torch.nn as nn

import torchvision
from torchvision import datasets, transforms, utils

import warnings
warnings.filterwarnings('ignore')

In [12]:
#Creating the Parameters

parameters = {'ngf':32,
              'ndf':64,
              'num_epochs':100,
              'decay_epoch': 100,
              'lgG':0.0002,
              'lgD':0.0002,
              'beta1':.5,
              'beta2':.9999,
              'lambdaA':10,
              'lambdaB':10,
              'batch_size':32,
              'pool_size':50, 
              'img_width':256, 
              'img_height':256, 
              'img_depth':3,
              'input_size':128,
              'resize_size':128,
              'crop_size':256,
              'fliplr':True,
              'num_resnet':6,
              'num_workers':2
              }
          

In [13]:
class ConvBlock(torch.nn.Module):
    
    #Initialize a dunction
    def __init__(self, input_size, output_size, kernel_size=3, stride=2,
                 padding=1, activation='relu', batch_norm=True):
        super(ConvBlock,self).__init__()
        
        self.conv = torch.nn.Conv2d(input_size, output_size, kernel_size,
                                    stride, padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.relu = torch.nn.ReLU(True)
        self.lrelu = torch.nn.LeakyReLU(0.2,True)
        self.tanh = torch.nn.Tanh()
    
    #Defining the forward function
    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.conv(x))
        else:
            out = self.conv(x)
        
        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out
  

In [14]:
class DeconvBlock(torch.nn.Module):

    #Initialization Function
    def __init__(self, input_size, output_size, kernel_size=3, stride=2,
                 padding=1, output_padding=1, activation='relu',
                 batch_norm=True):
      
        super(DeconvBlock,self).__init__()

        self.deconv = torch.nn.ConvTranspose2d(input_size, output_size,
                                               kernel_size, stride, padding,
                                               output_padding)
        self.batch_norm = batch_norm
        self.bn = torch.nn.InstanceNorm2d(output_size)
        self.activation = activation
        self.relu = torch.nn.ReLU(True)
    
    #Forward pass
    def forward(self,x):
        if self.batch_norm:
            out = self.bn(self.deconv(x))
        else:
            out = self.deconv(x)
        if self.activation == 'relu':
            return self.relu(out)
        elif self.activation == 'lrelu':
            return self.lrelu(out)
        elif self.activation == 'tanh':
            return self.tanh(out)
        elif self.activation == 'no_act':
            return out

In [15]:
class ResnetBlock(torch.nn.Module):
    def __init__(self,num_filter,kernel_size=3,stride=1,padding=0):
        super(ResnetBlock,self).__init__()

        conv1 = torch.nn.Conv2d(num_filter, num_filter, kernel_size,
                                stride, padding)
        
        conv2 = torch.nn.Conv2d(num_filter, num_filter, kernel_size,
                                stride, padding)
        
        bn = torch.nn.InstanceNorm2d(num_filter)
        relu = torch.nn.ReLU(True)
        pad = torch.nn.ReflectionPad2d(1)
        
        self.resnet_block = torch.nn.Sequential(
            pad,
            conv1,
            bn,
            relu,
            pad,
            conv2,
            bn
            )
    def forward(self,x):
        out = self.resnet_block(x)
        return out

In [16]:
class Generator(torch.nn.Module):
    def __init__(self,input_dim,num_filter,output_dim,num_resnet):
        super(Generator,self).__init__()
        
        #Reflection padding
        self.pad = torch.nn.ReflectionPad2d(3)
        
        #Encoder
        # Input Layer - NN Layer 1
        self.conv1 = ConvBlock(input_dim, num_filter, kernel_size=7, stride=1, 
                               padding=0)
        
        # NN Layer 2
        self.conv2 = ConvBlock(num_filter, num_filter*2)
        
        ## NN Layer 3
        self.conv3 = ConvBlock(num_filter*2, num_filter*4)
        
        #Transformer
        self.resnet_blocks = []
        for i in range(num_resnet):
            self.resnet_blocks.append(ResnetBlock(num_filter*4))
        self.resnet_blocks = torch.nn.Sequential(*self.resnet_blocks)
        
        #Decoder
        self.deconv1 = DeconvBlock(num_filter*4, num_filter*2)
        self.deconv2 = DeconvBlock(num_filter*2, num_filter)
        self.deconv3 = ConvBlock(num_filter, output_dim, kernel_size=7,
                                 stride=1, padding=0, activation='tanh',
                                 batch_norm=False)
    
    #Forward Function
    def forward(self,x):
        
        #Encoder
        enc1 = self.conv1(self.pad(x))
        enc2 = self.conv2(enc1)
        enc3 = self.conv3(enc2)
        
        #Transformer
        res = self.resnet_blocks(enc3)
        
        #Decoder
        dec1 = self.deconv1(res)
        dec2 = self.deconv2(dec1)
        out = self.deconv3(self.pad(dec2))
        return out
    

In [17]:
class Discriminator(torch.nn.Module):
    def __init__(self,input_dim,num_filter,output_dim):
        super(Discriminator,self).__init__()

        #Input - NN Layer 1
        conv1 = ConvBlock(input_dim,num_filter,kernel_size=4,stride=2,padding=1,activation='lrelu',batch_norm=False)
        
        #NN Layer 2
        conv2 = ConvBlock(num_filter,num_filter*2,kernel_size=4,stride=2,padding=1,activation='lrelu')
        
        #NN Layer 3
        conv3 = ConvBlock(num_filter*2,num_filter*4,kernel_size=4,stride=2,padding=1,activation='lrelu')
        
        #NN Layer 4
        conv4 = ConvBlock(num_filter*4,num_filter*8,kernel_size=4,stride=1,padding=1,activation='lrelu')
        
        #Output - NN Layer 5
        conv5 = ConvBlock(num_filter*8,output_dim,kernel_size=4,stride=1,padding=1,activation='no_act',batch_norm=False)
        
        
        self.conv_blocks = torch.nn.Sequential(
            conv1,
            conv2,
            conv3,
            conv4,
            conv5
            )
        
    def forward(self,x):
        out = self.conv_blocks(x)
        return out

    #Creating the weights for the Discriminator
    def normal_weight_init(self, mean=0.0, std=0.02):
        for m in self.children():
            if isinstance(m, ConvBlock):
                torch.nn.init.normal_(m.conv.weight.data, mean, std)


In [18]:
Generator_A = Generator(3, parameters['ngf'], 3, 
                        parameters['num_resnet']).cuda()
Generator_B = Generator(3, parameters['ngf'], 3, 
                        parameters['num_resnet']).cuda()

Discriminator_A = Discriminator(3, parameters['ndf'], 1).cuda()
Discriminator_B = Discriminator(3, parameters['ndf'], 1).cuda()

In [19]:
Generator_optimizer = torch.optim.Adam(itertools.chain(Generator_A.parameters(),
                                                       Generator_B.parameters()),
                                                       betas=(parameters['beta1'], parameters['beta2']),
                                                       lr=parameters['lgG'])

Discriminator_A_optimizer = torch.optim.Adam(itertools.chain(Discriminator_A.parameters(),
                                                           Discriminator_B.parameters()),
                                                           betas=(parameters['beta1'], parameters['beta2']),
                                                           lr=parameters['lgD'])

Discriminator_B_optimizer = torch.optim.Adam(itertools.chain(Discriminator_A.parameters(),
                                                           Discriminator_B.parameters()),
                                                           betas=(parameters['beta1'], parameters['beta2']),
                                                           lr=parameters['lgD'])


## Loading The Models and Optimizer

In [20]:
Generator_A_pth = 'Generator_A.pth'
path = f'/content/drive/My Drive/Thinkful/CapstoneIV/Saved_Models/Generator_A.pth'
Generator_A.load_state_dict(torch.load(path))

<All keys matched successfully>

In [21]:
Generator_B_pth = 'Generator_A.pth'
path = f'/content/drive/My Drive/Thinkful/CapstoneIV/Saved_Models/Generator_B.pth'
Generator_B.load_state_dict(torch.load(path))

<All keys matched successfully>

In [22]:
Discriminator_A_pth = 'Discriminator_A.pth'
path = f'/content/drive/My Drive/Thinkful/CapstoneIV/Saved_Models/Discriminator_A.pth'
Discriminator_A.load_state_dict(torch.load(path))

<All keys matched successfully>

In [23]:
Discriminator_B_pth = 'Discriminator_B.pth'
path = f'/content/drive/My Drive/Thinkful/CapstoneIV/Saved_Models/Discriminator_B.pth'
Discriminator_B.load_state_dict(torch.load(path))

<All keys matched successfully>

In [25]:
Generator_optimizer_pth = 'Generator_optimizer.pth'
path = f'/content/drive/My Drive/Thinkful/CapstoneIV/Saved_Models/Generator_optimizer.pth'
Generator_optimizer.load_state_dict(torch.load(path))